## Problema de mezclas: Mezcla de aceites
---
Una empresa elabora un cierto alimento refinando diferentes tipos de aceite y mezclándolos. Los aceites se clasifican en dos categorías: vegetales (VEG1 y VEG2) y no vegetales (OIL1, OIL2 y OIL3).
Cada uno de estos tipos, vegetal o no vegetal, requiere una línea de producción diferente para el refinado:
* La capacidad máxima de refino es de 250 Tn de aceite vegetal y 300 Tn de no vegetal.
* Se puede asumir que el coste de refino es nulo y que durante el proceso no se producen pérdidas de peso.

Cada tonelada de producto final se vende a un precio de 15 EUR.
La siguiente tabla muestra el coste (por tonelada en EUR) de cada tipo de aceite:

|       | VEG1 | VEG2 | OIL1 | OIL2 | OIL3 |
|-------|------|------|------|------|------|
| Coste |  11  |  12  |  13  |  11  | 11.5 |


Por restricciones tecnológicas, la dureza del producto final debe estar entre 4 y 9 unidades. Se puede asumir que la dureza se mezcla linealmente. Esto es, la dureza final es la media ponderada de la dureza de los distintos aceites utilizados en la mezcla.

La siguiente tabla muestra la dureza de cada tipo de aceite:

|       | VEG1 | VEG2 | OIL1 | OIL2 | OIL3 |
|-------|------|------|------|------|------|
| Dureza|  7.2 |  6.3 |  2.0 |  5.2 |  4.0 |

**Determinar qué cantidad de aceite de cada tipo debe usarse en la mezcla para MAXIMIZAR el beneficio obtenido** (diferencia entre el precio de venta de la mezcla y el precio de compra de los aceites).

In [ ]:
import gurobipy as grb

In [ ]:
# --- Datos del Problema ---
# Usamos un diccionario para indexar por el nombre del aceite

Aceites = ["VEG1", "VEG2", "OIL1", "OIL2", "OIL3"]
Tipos = {
    "VEG1": "V",
    "VEG2": "V",
    "OIL1": "N",
    "OIL2": "N",
    "OIL3": "N",
}  # V=Vegetal, N=No Vegetal

Costos = [11.0, 12.0, 13.0, 11.0, 11.5]
Dureza = [7.2, 6.3, 2.0, 5.2, 4.0]
Venta_Precio = 15.0

Capacidad_V = 250
Capacidad_N = 300
Dureza_Min = 4.0
Dureza_Max = 6.0

print(f"La lista Aceites es: {Aceites}")

La lista Aceites es: ['VEG1', 'VEG2', 'OIL1', 'OIL2', 'OIL3']


In [ ]:
# --- MODELO GUROBI ---
m = grb.Model("MezclaAceites")

# 1. Variables: x[i] = Cantidad (Tn) del aceite 'i'
# Usamos Aceites como índice (más legible y evita errores de índice)
x = m.addVars(Aceites, vtype=grb.GRB.CONTINUOUS, name="X")

# 2. Función Objetivo: Maximizar Beneficio
# El índice 'i' aquí es el nombre del aceite, y usamos enumerate para obtener el índice de las listas
m.setObjective(
    grb.quicksum(
        x[Aceites[i]] * (Venta_Precio - Costos[i]) for i in range(len(Aceites))
    ),
    grb.GRB.MAXIMIZE,
)

In [ ]:
# 3. Restricciones de Capacidad (Refinado)
# a) Capacidad Aceite Vegetal (VEG1 y VEG2)
m.addConstr(x["VEG1"] + x["VEG2"] <= Capacidad_V, "Capacidad_Vegetal")
# b) Capacidad Aceite No Vegetal (OIL1, OIL2, OIL3)
m.addConstr(x["OIL1"] + x["OIL2"] + x["OIL3"] <= Capacidad_N, "Capacidad_NoVegetal")
# Nota: La capacidad de cada tipo es de 250 Tn/300 Tn, NO la suma de TODAS las variables.

# 4. Restricciones de Dureza (Linealizadas)
# Suma total de aceite producido: Total_X = sum(x[i])
Total_X = grb.quicksum(x[i] for i in Aceites)
# Suma ponderada de la dureza: Sum_Hard = sum(Hardness[i] * x[i])
Sum_Hard = grb.quicksum(Dureza[i] * x[Aceites[i]] for i in range(len(Aceites)))

# a) Dureza Mínima (Sum_Hard >= Dureza_Min * Total_X)
# Linealizado: Sum( (Hardness[i] - Dureza_Min) * x[i] ) >= 0
m.addConstr(
    grb.quicksum((Dureza[i] - Dureza_Min) * x[Aceites[i]] for i in range(len(Aceites)))
    >= 0,
    "Dureza_Minima",
)

# b) Dureza Máxima (Sum_Hard <= Dureza_Max * Total_X)
# Linealizado: Sum( (Dureza_Max - Hardness[i]) * x[i] ) >= 0
m.addConstr(
    grb.quicksum((Dureza_Max - Dureza[i]) * x[Aceites[i]] for i in range(len(Aceites)))
    >= 0,
    "Dureza_Maxima",
)

<gurobi.Constr *Awaiting Model Update*>

In [48]:
# Optimizar y Mostrar Resultados
m.optimize()

if m.status == grb.GRB.OPTIMAL:
    print(f"Beneficio Máximo: {m.ObjVal:.2f} EUR")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 14 rows, 5 columns and 45 nonzeros
Coefficient statistics:
  Matrix range     [3e-01, 4e+00]
  Objective range  [2e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 3e+02]

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.175000000e+03
Beneficio Máximo: 2175.00 EUR
